In [ ]:
import mercury as mr
import requests
import time

In [ ]:
json_url = "https://library.iit.edu/api/v1/slides_rest"
hours_url = "https://iit.libcal.com/api_hours_today.php?iid=5544&lid=0&format=json&systemTime=0"
template_filepath = "template.html"
cta_flag = True
build_html = ""

In [ ]:
def dru_slides_builder(json_url):
    slides_call = requests.get(json_url)
    if slides_call.status_code == 200:
        web_slides_dict = slides_call.json()

    slides_list = []
    slides = ''

    for i in web_slides_dict:
        image_url = i["field_image"]
        title = i['title_1']
        if image_url != '':
            section = f'<section data-background-image={image_url} data-background-size="contain, 100%"></section>'
            slides_list.append(section)

    for i in slides_list:
        slides += ("\t\t\t\t\t\t" + i + "\n")

    slides = "<div class=\"slides\">\n" + slides[:-1]

    return slides

In [ ]:
def build_slides_html(template_filepath, slides, hours_url, cta_flag):

    hours_call = requests.get(hours_url)
    if hours_call.status_code == 200:
        hours_dict = hours_call.json()

    with open(template_filepath, 'r') as template:
        slides_html = template.read()

    content = "<div class=\"slides\">"

    hours_div = '<div id="todays_hours">'

    openfrom = hours_dict['locations'][0]['times']['hours'][0]['from']
    opento = hours_dict['locations'][0]['times']['hours'][0]['to']
    hours_statement = '''<div id=\"todays_hours\">
                <p>{todays_hours}</p>
            '''

    todays_hours = "Today's Library Hours: {openfrom} - {opento}"
    todays_hours = todays_hours.format(openfrom=openfrom, opento=opento)
    hours_statement = hours_statement.format(todays_hours=todays_hours)

    slides_html = slides_html.replace(content, slides)

    if cta_flag == False:
        sidebar_right = "<div id=\"sidebar-right\">"
        sidebar_right_hidden = "<div id=\"sidebar-right\" class=\"hidden\">"
        slides_html = slides_html.replace(sidebar_right, sidebar_right_hidden)

    slides_html = slides_html.replace(hours_div, hours_statement)

    return slides_html

In [ ]:
app = mr.App(title="Slides Builder", description="App for generating an html file from images hosted in Drupal to create a Reveal.js digital sign.")

In [ ]:
my_dir = mr.OutputDir()

In [ ]:
my_flag = mr.Checkbox(value=True, label="Include CTA Tracker?")
show_html = mr.Checkbox(value=False, label="Show raw HTML after generating?")
button = mr.Button(label="Click To Build Slides", style="primary")

# Digital Signage Builder
## IIT Galvin Library
#### Click the button at left to generate a new index.html file containing updated digital slide content.
#### Slide content is retrieved from Drupal.
#### Click "Output Files" at left to find the newly created index.html file.
#### Save this file in your Reveal.js directory, replacing the existing index.html.

In [ ]:
if button.clicked:
    slides = dru_slides_builder(json_url)
    
    if my_flag.value:
        cta_flag = True
    else:
        cta_flag = False

    build_html = build_slides_html(template_filepath, slides, hours_url, cta_flag)

    if show_html.value:
        print(build_html)
    else:
        pass

    with open(os.path.join(my_dir.path, "index.html"), "w") as fout:
        fout.write(build_html)
        print('''New Slide Page Created! Check the "Output Files" folder.''')